In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from eq_odds_postprocessing import EqOddsPostprocessing
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso
from random import sample

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## Huangrui's Dataset 

In [2]:
default_mappings = {
    #Huangrui flip the lable
    'label_maps': [{0: 'Did recid.', 1: 'No recid.'}],
    'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'},
                                 {1.0: 'Caucasian', 0.0: 'Not Caucasian'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class CompasDataset(StandardDataset):
    """ProPublica COMPAS Dataset.

    See :file:`aif360/data/raw/compas/README.md`.
    """

    def __init__(self, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[],
                 features_to_drop=[], na_values=[],
                 custom_preprocessing=None,
                 metadata=default_mappings,
                 path='./Huangrui/recidivism/recidivism_test1.csv'):

    
        df = pd.read_csv(path,index_col=False)
        df.rename(columns={'Probationerssex_Female': 'sex'}, inplace=True)
        df.drop(["Probationerssex_Male","Probationerssex_Notascertained"], axis=1, inplace=True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        #flip the Y lable to 0: recid, 1: no recid
        df["Y"] = [1 if x == 0 else 0 for x in df["Y"]]
        
        super(CompasDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


#### Load dataset and specify options

In [3]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [4]:
experiments_info = {}
budget = 1
for K in range(1, 6):
    # dataset_orig_train= Zindi(path="./Huangrui/zindi/zindi_train{}.csv".format(K),protected_attribute_names=['sex'],
    #             privileged_classes=[[1]])
    dataset_orig_test= CompasDataset(path="./Huangrui/recidivism/recidivism_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    # #only use the budget% of the training data
    # dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)
    # Lasso linear classifier and predictions
    lmod = pickle.load(open('experiments/recidivism'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    data= pickle.load(open("./experiments/recidivism{}_sex.pkl".format(K), "rb"))
    X_test = data["Xtest"]
    X_test["Probationerssex_Female"] = [1]*X_test.shape[0]
    y_test = dataset_orig_test_pred.labels
    y_test_pred = lmod.predict(X_test)
    #flip the lable
    dataset_orig_test_pred.labels = (y_test_pred<0.5).reshape(-1,1)
    metric_test = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                                  unprivileged_groups, privileged_groups)
   
    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_orig_test_pred.labels), "Equal opportunity difference": metric_test["Equal opportunity difference"]}

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6625
Statistical parity difference = -0.0678
Disparate impact = 0.9205
Average odds difference = -0.0357
Equal opportunity difference = 0.0070
Theil index = 0.1032
K = 1, budget = 1
The Error for the test dataset is 0.2794
The Equal opportunity difference for the test dataset is 0.007031


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6982
Statistical parity difference = -0.1000
Disparate impact = 0.8792
Average odds difference = -0.0409
Equal opportunity difference = -0.0475
Theil index = 0.1148
K = 2, budget = 1
The Error for the test dataset is 0.2572
The Equal opportunity difference for the test dataset is -0.04748


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6805
Statistical parity difference = -0.0840
Disparate impact = 0.8646
Average odds difference = -0.0541
Equal opportunity difference = -0.0004
Theil index = 0.2553
K = 3, budget = 1
The Error for the test dataset is 0.3182
The Equal opportunity difference for the test dataset is -0.0004472


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Balanced accuracy = 0.6881
Statistical parity difference = -0.0916
Disparate impact = 0.8863
Average odds difference = -0.0583
Equal opportunity difference = -0.0128
Theil index = 0.1312
K = 4, budget = 1
The Error for the test dataset is 0.2705
The Equal opportunity difference for the test dataset is -0.01282
Balanced accuracy = 0.7064
Statistical parity difference = -0.0599
Disparate impact = 0.9253
Average odds difference = -0.0200
Equal opportunity difference = -0.0238
Theil index = 0.1062
K = 5, budget = 1
The Error for the test dataset is 0.2479
The Equal opportunity difference for the test dataset is -0.0238


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [5]:
experiments_info

{'K = 1, budget = 1': {'Error': 0.2793991416309013,
  'Equal opportunity difference': 0.007031126037975399},
 'K = 2, budget = 1': {'Error': 0.25719192786603695,
  'Equal opportunity difference': -0.047482136337557934},
 'K = 3, budget = 1': {'Error': 0.31816230141691715,
  'Equal opportunity difference': -0.00044724098578086124},
 'K = 4, budget = 1': {'Error': 0.270502361528553,
  'Equal opportunity difference': -0.012823996087594325},
 'K = 5, budget = 1': {'Error': 0.24785223367697595,
  'Equal opportunity difference': -0.02380097879282228}}